In [1]:
import pandas as pd
import pickle

import re
import string
import fuzzymatcher

from pymongo import MongoClient

In [2]:
client = MongoClient()
db = client.genius
db.list_collection_names()

['artist_ids', 'song_urls', 'song_lyrics']

## Adding Artists to Lyrics (Genius)

In [3]:
cursor = db.song_lyrics.find({}, {'_id':0, 'song_id':1, 'song_title':1, 'song_url':1, 'raw_lyrics':1})
genius_df = pd.DataFrame(list(cursor))
genius_df.head()

,raw_lyrics,song_id,song_title,song_url
0,"\n\nOooh, love\nYou're coming down on me\n\nOo...",987434,Berlin,https://genius.com/The-essex-green-berlin-lyrics
1,\n\nI just wanna remain an anonimity\nI don't ...,1009076,Big Green Tree,https://genius.com/The-essex-green-big-green-t...
2,\n\nA moment taken slowly by the sea\nWind blo...,1551187,By the Sea,https://genius.com/The-essex-green-by-the-sea-...
3,"\n\nI lie\nHalfway to sleep, sleep so restless...",1745373,Carballo,https://genius.com/The-essex-green-carballo-ly...
4,"\n\nYou were down when I met you, but\nDeep in...",1073422,Chartiers,https://genius.com/The-essex-green-chartiers-l...


In [4]:
genius_df.drop_duplicates('song_id',inplace=True)

In [5]:
genius_df.shape

(52036, 4)

In [6]:
cursor = db.song_urls.find({}, {'_id':0, 'artist_id':1, 'id':1})
artist_id_df = pd.DataFrame(list(cursor))
artist_id_df.columns = ['artist_id','song_id']
artist_id_df.head()

,artist_id,song_id
0,358098,987434
1,358098,1009076
2,358098,1551187
3,358098,1745373
4,358098,1073422


In [7]:
artist_id_df.drop_duplicates('song_id',inplace=True)

In [8]:
genius_df = genius_df.merge(artist_id_df, on='song_id', how='left')
genius_df.head()

,raw_lyrics,song_id,song_title,song_url,artist_id
0,"\n\nOooh, love\nYou're coming down on me\n\nOo...",987434,Berlin,https://genius.com/The-essex-green-berlin-lyrics,358098
1,\n\nI just wanna remain an anonimity\nI don't ...,1009076,Big Green Tree,https://genius.com/The-essex-green-big-green-t...,358098
2,\n\nA moment taken slowly by the sea\nWind blo...,1551187,By the Sea,https://genius.com/The-essex-green-by-the-sea-...,358098
3,"\n\nI lie\nHalfway to sleep, sleep so restless...",1745373,Carballo,https://genius.com/The-essex-green-carballo-ly...,358098
4,"\n\nYou were down when I met you, but\nDeep in...",1073422,Chartiers,https://genius.com/The-essex-green-chartiers-l...,358098


In [9]:
cursor = db.artist_ids.find({}, {'_id':0, 'artist_name':1, 'artist_id':1})
artist_name_df = pd.DataFrame(list(cursor))
artist_name_df.head()

,artist_id,artist_name
0,358098,The Essex Green
1,63636,FKA twigs
2,1602422,The Margarets
3,353058,Los Planetas
4,21765,Hayley Kiyoko


In [10]:
artist_name_df.drop_duplicates('artist_id',inplace=True)
artist_name_df.shape

(531, 2)

In [11]:
genius_df = genius_df.merge(artist_name_df, on='artist_id')
genius_df.head()

,raw_lyrics,song_id,song_title,song_url,artist_id,artist_name
0,"\n\nOooh, love\nYou're coming down on me\n\nOo...",987434,Berlin,https://genius.com/The-essex-green-berlin-lyrics,358098,The Essex Green
1,\n\nI just wanna remain an anonimity\nI don't ...,1009076,Big Green Tree,https://genius.com/The-essex-green-big-green-t...,358098,The Essex Green
2,\n\nA moment taken slowly by the sea\nWind blo...,1551187,By the Sea,https://genius.com/The-essex-green-by-the-sea-...,358098,The Essex Green
3,"\n\nI lie\nHalfway to sleep, sleep so restless...",1745373,Carballo,https://genius.com/The-essex-green-carballo-ly...,358098,The Essex Green
4,"\n\nYou were down when I met you, but\nDeep in...",1073422,Chartiers,https://genius.com/The-essex-green-chartiers-l...,358098,The Essex Green


In [12]:
genius_df.shape

(52036, 6)

#### Removing non-queried artists

In [13]:
returned_artists = genius_df.artist_name.unique().tolist()

In [14]:
pickle_in = open('../Data/indie_pop_artists','rb')
queried_artists = pickle.load(pickle_in)
pickle_in.close()

In [15]:
print(len(queried_artists))
print(len(returned_artists))
print(len(set(queried_artists).intersection(set(returned_artists))))

716
530
530


In [16]:
artists_df = pd.DataFrame(list(set(queried_artists).intersection(set(returned_artists))))
artists_df.columns = ['artist_name']
artists_df.head()

,artist_name
0,Clap Your Hands Say Yeah
1,The Ruby Suns
2,Sex Clark Five
3,Glass Animals
4,Sodagreen


In [17]:
genius_df = genius_df.merge(artists_df,on='artist_name',how='inner')
genius_df.head()

,raw_lyrics,song_id,song_title,song_url,artist_id,artist_name
0,"\n\nOooh, love\nYou're coming down on me\n\nOo...",987434,Berlin,https://genius.com/The-essex-green-berlin-lyrics,358098,The Essex Green
1,\n\nI just wanna remain an anonimity\nI don't ...,1009076,Big Green Tree,https://genius.com/The-essex-green-big-green-t...,358098,The Essex Green
2,\n\nA moment taken slowly by the sea\nWind blo...,1551187,By the Sea,https://genius.com/The-essex-green-by-the-sea-...,358098,The Essex Green
3,"\n\nI lie\nHalfway to sleep, sleep so restless...",1745373,Carballo,https://genius.com/The-essex-green-carballo-ly...,358098,The Essex Green
4,"\n\nYou were down when I met you, but\nDeep in...",1073422,Chartiers,https://genius.com/The-essex-green-chartiers-l...,358098,The Essex Green


In [18]:
genius_df.columns = ['raw_lyrics', 'genius_song_id', 'song_title', 
                     'genius_song_url', 'genius_artist_id', 'artist_name']

In [19]:
genius_df.shape

(52036, 6)

In [20]:
genius_df.artist_name.nunique()

530

### Put it to a pickle!

In [21]:
genius_df.to_pickle('../Data/lyrics_with_artists')

## Getting Spotify artists and songs

In [22]:
client = MongoClient()
db = client.spotify
db.list_collection_names()

['track_features', 'tracks', 'artists', 'albums']

In [23]:
cursor = db.tracks.find({}, 
    {'_id':0, 'album_uri':1, 'artist_id':1, 'artist_name':1, 'artist_uri':1, 'duration_ms':1, 'explicit':1, 'id':1, 'name':1, 'track_number':1, 'uri':1, 'song_spotify_page':1})
spotify_df = pd.DataFrame(list(cursor))
spotify_df.columns = ['album_uri', 'artist_id', 'artist_name', 'artist_uri', 'duration_ms',
       'explicit', 'id', 'song_title', 'song_spotify_page', 'track_number', 'uri']
spotify_df.head()

,album_uri,artist_id,artist_name,artist_uri,duration_ms,explicit,id,song_title,song_spotify_page,track_number,uri
0,spotify:album:2h6MSR1rWemHOtmurgRq3T,7kAKqNxPBkfjgdHzUIdBtI,Suburban Kids With Biblical Names,spotify:artist:7kAKqNxPBkfjgdHzUIdBtI,138626,False,4okEZakOVppAtP4Dawd52x,Marry Me,https://open.spotify.com/track/4okEZakOVppAtP4...,1,spotify:track:4okEZakOVppAtP4Dawd52x
1,spotify:album:2h6MSR1rWemHOtmurgRq3T,7kAKqNxPBkfjgdHzUIdBtI,Suburban Kids With Biblical Names,spotify:artist:7kAKqNxPBkfjgdHzUIdBtI,187106,False,2LV6sB5zTsu0R5r5kWohlD,Loop Duplicate My Heart,https://open.spotify.com/track/2LV6sB5zTsu0R5r...,2,spotify:track:2LV6sB5zTsu0R5r5kWohlD
2,spotify:album:2h6MSR1rWemHOtmurgRq3T,7kAKqNxPBkfjgdHzUIdBtI,Suburban Kids With Biblical Names,spotify:artist:7kAKqNxPBkfjgdHzUIdBtI,192000,False,2RPcqvy4bdznms2eHU6vnE,A Couple Of Instruments,https://open.spotify.com/track/2RPcqvy4bdznms2...,3,spotify:track:2RPcqvy4bdznms2eHU6vnE
3,spotify:album:2h6MSR1rWemHOtmurgRq3T,7kAKqNxPBkfjgdHzUIdBtI,Suburban Kids With Biblical Names,spotify:artist:7kAKqNxPBkfjgdHzUIdBtI,176026,False,53uzYuKe433aXBjzBiuvqe,Parakit,https://open.spotify.com/track/53uzYuKe433aXBj...,4,spotify:track:53uzYuKe433aXBjzBiuvqe
4,spotify:album:2h6MSR1rWemHOtmurgRq3T,7kAKqNxPBkfjgdHzUIdBtI,Suburban Kids With Biblical Names,spotify:artist:7kAKqNxPBkfjgdHzUIdBtI,198013,False,7p0pJgizlHS5msrgDPU6li,Trees And Squirrels,https://open.spotify.com/track/7p0pJgizlHS5msr...,5,spotify:track:7p0pJgizlHS5msrgDPU6li


#### Removing non-queried artists

In [24]:
returned_artists = spotify_df.artist_name.unique().tolist()

In [25]:
print(len(queried_artists))
print(len(returned_artists))
print(len(set(queried_artists).intersection(set(returned_artists))))

716
714
539


In [26]:
artists_df = pd.DataFrame(list(set(queried_artists).intersection(set(returned_artists))))
artists_df.columns = ['artist_name']
artists_df.head()

,artist_name
0,Clap Your Hands Say Yeah
1,The Ruby Suns
2,Sex Clark Five
3,Glass Animals
4,Sodagreen


In [27]:
spotify_df = spotify_df.merge(artists_df,on='artist_name')
spotify_df.head()

,album_uri,artist_id,artist_name,artist_uri,duration_ms,explicit,id,song_title,song_spotify_page,track_number,uri
0,spotify:album:2h6MSR1rWemHOtmurgRq3T,7kAKqNxPBkfjgdHzUIdBtI,Suburban Kids With Biblical Names,spotify:artist:7kAKqNxPBkfjgdHzUIdBtI,138626,False,4okEZakOVppAtP4Dawd52x,Marry Me,https://open.spotify.com/track/4okEZakOVppAtP4...,1,spotify:track:4okEZakOVppAtP4Dawd52x
1,spotify:album:2h6MSR1rWemHOtmurgRq3T,7kAKqNxPBkfjgdHzUIdBtI,Suburban Kids With Biblical Names,spotify:artist:7kAKqNxPBkfjgdHzUIdBtI,187106,False,2LV6sB5zTsu0R5r5kWohlD,Loop Duplicate My Heart,https://open.spotify.com/track/2LV6sB5zTsu0R5r...,2,spotify:track:2LV6sB5zTsu0R5r5kWohlD
2,spotify:album:2h6MSR1rWemHOtmurgRq3T,7kAKqNxPBkfjgdHzUIdBtI,Suburban Kids With Biblical Names,spotify:artist:7kAKqNxPBkfjgdHzUIdBtI,192000,False,2RPcqvy4bdznms2eHU6vnE,A Couple Of Instruments,https://open.spotify.com/track/2RPcqvy4bdznms2...,3,spotify:track:2RPcqvy4bdznms2eHU6vnE
3,spotify:album:2h6MSR1rWemHOtmurgRq3T,7kAKqNxPBkfjgdHzUIdBtI,Suburban Kids With Biblical Names,spotify:artist:7kAKqNxPBkfjgdHzUIdBtI,176026,False,53uzYuKe433aXBjzBiuvqe,Parakit,https://open.spotify.com/track/53uzYuKe433aXBj...,4,spotify:track:53uzYuKe433aXBjzBiuvqe
4,spotify:album:2h6MSR1rWemHOtmurgRq3T,7kAKqNxPBkfjgdHzUIdBtI,Suburban Kids With Biblical Names,spotify:artist:7kAKqNxPBkfjgdHzUIdBtI,198013,False,7p0pJgizlHS5msrgDPU6li,Trees And Squirrels,https://open.spotify.com/track/7p0pJgizlHS5msr...,5,spotify:track:7p0pJgizlHS5msrgDPU6li


In [28]:
spotify_df.columns = ['spotify_album_uri', 'spotify_artist_id', 'artist_name', 'spotify_artist_uri', 'duration_ms',
                      'explicit', 'spotify_song_id', 'song_title', 'song_spotify_page', 'track_number', 'spotify_song_uri']

In [29]:
spotify_df.shape

(35538, 11)

In [30]:
spotify_df.artist_name.nunique()

539

## Combining Genius and Spotify

Let's establish a baseline for our matching

In [31]:
pd.merge(spotify_df, genius_df, how='inner', on=['artist_name','song_title']).shape

(16588, 15)

Making everything lowercase but also preserving a version of the Song Title and Artist name so we have clean versions for laster use.

In [32]:
spotify_df['spotify_song_title'] = spotify_df.song_title
spotify_df['spotify_artist_name'] = spotify_df.artist_name

spotify_df['artist_name'] = spotify_df.artist_name.apply(lambda x: x.lower())
spotify_df['song_title'] = spotify_df.song_title.apply(lambda x: x.lower())
genius_df['artist_name'] = genius_df.artist_name.apply(lambda x: x.lower())
genius_df['song_title'] = genius_df.song_title.apply(lambda x: x.lower())

In [33]:
pd.merge(spotify_df, genius_df, how='inner', on=['artist_name','song_title']).shape

(19322, 17)

Removing numbers, symbols, punctuation, and extra spaces

In [34]:
alphabet = lambda x: re.sub('[\W]+|[0-9]+', ' ', x)
punctuation = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x)
extra_space = lambda x: re.sub(' +', ' ', x.strip())

spotify_df['artist_name'] = spotify_df.artist_name.apply(alphabet).apply(punctuation).apply(extra_space)
spotify_df['song_title'] = spotify_df.song_title.apply(alphabet).apply(punctuation).apply(extra_space)
genius_df['artist_name'] = genius_df.artist_name.apply(alphabet).apply(punctuation).apply(extra_space)
genius_df['song_title'] = genius_df.song_title.apply(alphabet).apply(punctuation).apply(extra_space)

We didn't manage to get all of our songs to match. Perhaps we can come back later if we have time.

In [35]:
pd.merge(spotify_df, genius_df, how='inner', on=['artist_name','song_title']).shape

(20456, 17)

In [36]:
combined_genius_spotify_uncleaned = pd.merge(spotify_df, genius_df, how='inner', on=['artist_name','song_title'])

In [37]:
combined_genius_spotify_uncleaned.to_pickle('../Data/combined_genius_spotify_uncleaned')

In [38]:
combined_genius_spotify_uncleaned.head()

,spotify_album_uri,spotify_artist_id,artist_name,spotify_artist_uri,duration_ms,explicit,spotify_song_id,song_title,song_spotify_page,track_number,spotify_song_uri,spotify_song_title,spotify_artist_name,raw_lyrics,genius_song_id,genius_song_url,genius_artist_id
0,spotify:album:2h6MSR1rWemHOtmurgRq3T,7kAKqNxPBkfjgdHzUIdBtI,suburban kids with biblical names,spotify:artist:7kAKqNxPBkfjgdHzUIdBtI,138626,False,4okEZakOVppAtP4Dawd52x,marry me,https://open.spotify.com/track/4okEZakOVppAtP4...,1,spotify:track:4okEZakOVppAtP4Dawd52x,Marry Me,Suburban Kids With Biblical Names,\n\nAny old chance I get I'm gonna marry you\n...,861607,https://genius.com/Suburban-kids-with-biblical...,353411
1,spotify:album:2h6MSR1rWemHOtmurgRq3T,7kAKqNxPBkfjgdHzUIdBtI,suburban kids with biblical names,spotify:artist:7kAKqNxPBkfjgdHzUIdBtI,187106,False,2LV6sB5zTsu0R5r5kWohlD,loop duplicate my heart,https://open.spotify.com/track/2LV6sB5zTsu0R5r...,2,spotify:track:2LV6sB5zTsu0R5r5kWohlD,Loop Duplicate My Heart,Suburban Kids With Biblical Names,\n\nAnd it's bigger than everything I have eve...,980120,https://genius.com/Suburban-kids-with-biblical...,353411
2,spotify:album:2h6MSR1rWemHOtmurgRq3T,7kAKqNxPBkfjgdHzUIdBtI,suburban kids with biblical names,spotify:artist:7kAKqNxPBkfjgdHzUIdBtI,176026,False,53uzYuKe433aXBjzBiuvqe,parakit,https://open.spotify.com/track/53uzYuKe433aXBj...,4,spotify:track:53uzYuKe433aXBjzBiuvqe,Parakit,Suburban Kids With Biblical Names,\n\nI'm going back to the place I was born\nMy...,1583231,https://genius.com/Suburban-kids-with-biblical...,353411
3,spotify:album:2h6MSR1rWemHOtmurgRq3T,7kAKqNxPBkfjgdHzUIdBtI,suburban kids with biblical names,spotify:artist:7kAKqNxPBkfjgdHzUIdBtI,198013,False,7p0pJgizlHS5msrgDPU6li,trees and squirrels,https://open.spotify.com/track/7p0pJgizlHS5msr...,5,spotify:track:7p0pJgizlHS5msrgDPU6li,Trees And Squirrels,Suburban Kids With Biblical Names,\n\nThe trees are wild and undisputably beauti...,1340623,https://genius.com/Suburban-kids-with-biblical...,353411
4,spotify:album:2h6MSR1rWemHOtmurgRq3T,7kAKqNxPBkfjgdHzUIdBtI,suburban kids with biblical names,spotify:artist:7kAKqNxPBkfjgdHzUIdBtI,178040,False,1drw02VGWNxVtZuF2Qjp8e,funeral face,https://open.spotify.com/track/1drw02VGWNxVtZu...,6,spotify:track:1drw02VGWNxVtZuF2Qjp8e,Funeral Face,Suburban Kids With Biblical Names,"\n\nSaid ""I love you,""\nSaid ""I like you, and ...",1042112,https://genius.com/Suburban-kids-with-biblical...,353411


spotify_df['artist_and_song'] = spotify_df.artist_name + spotify_df.song_title
genius_df['artist_and_song'] = genius_df.artist_name + genius_df.song_title

from fuzzywuzzy import process


names_array=[]
ratio_array=[]
def match_names(wrong_names,correct_names):
    for row in wrong_names:
        x=process.extractOne(row, correct_names)
        names_array.append(x[0])
        ratio_array.append(x[1])
    return names_array,ratio_array


#Wrong country names dataset
wrong_names = genius_df['artist_and_song'].dropna().values

#Correct country names dataset
correct_names = spotify_df['artist_and_song'].values

name_match,ratio_match=match_names(wrong_names,correct_names)

spotify_df['correct_country_name']=pd.Series(name_match)
spotify_df['country_names_ratio']=pd.Series(ratio_match)


spotify_df[['artist_and_song','correct_country_name','country_names_ratio']].head(10)

spotify_df[['artist_and_song','correct_country_name','country_names_ratio']].head(10)

spotify_df[spotify_df.artist_and_song.str.contains('NEAR HERE END IS IS THE')]

pd.merge(spotify_df, genius_df, how='inner', on=['artist_name','song_title'])

import difflib 

genius_df.head()